# 14 - PREPARACIÓN DEL CÓDIGO DE PRODUCCIÓN Y ENTRENAMIENTO

## IMPORTAR LOS PAQUETES

In [34]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#Transformación de datos
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler

#Modelo
from sklearn.linear_model import RidgeCV

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

import cloudpickle
import joblib

## CARGAR LOS DATOS

In [36]:
df = pd.read_csv('../../02_Datos/03_Trabajo/trabajo.csv', index_col= 0).reset_index()
df.drop(columns= 'index', inplace= True)
df

,mpg,cilindros,desplazamiento,caballos,peso,aceleracion,any_modelo,origen,nombre_auto
0,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
1,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
2,15.0,8,429.0,198.0,4341.0,10.0,70,1,ford galaxie 500
3,14.0,8,454.0,220.0,4354.0,9.0,70,1,chevrolet impala
4,14.0,8,440.0,215.0,4312.0,8.5,70,1,plymouth fury iii
...,...,...,...,...,...,...,...,...,...
274,27.0,4,140.0,86.00,2790.0,15.6,82,1,ford mustang gl
275,44.0,4,97.0,52.00,2130.0,24.6,82,2,vw pickup
276,32.0,4,135.0,84.00,2295.0,11.6,82,1,dodge rampage
277,28.0,4,120.0,79.00,2625.0,18.6,82,1,ford ranger


## CALIDAD DE DATOS

In [37]:
#Cambiamos los nulos '?' por nan
df['caballos'] = df.caballos.replace({'?':np.nan})
#Cambiamos el tipo de variable
df = df.astype({'caballos':'float64',
                'cilindros':'object',
                'any_modelo':'object',
                'origen':'object'})

#cambiamos el nombre_auto
df.nombre_auto = df.nombre_auto.replace( {'vw rabbit':'volkswagen rabbit',
                        'vw dasher (diesel)':'volkswagen dasher Diesel',
                        'vw pickup':'volkswagen pickup'}
                      )
df.nombre_auto = df.nombre_auto.replace( {'chevy s-10':'chevrolet s-10',
                        'chevy c10':'chevrolet c10',
                        'chevy c20':'chevrolet c20'}
                      )

# Cargar CountVectorizer y lista de stop_words durante la producción
loaded_cv = joblib.load('../../09_Otros/count_vectorizer_model.pkl')
loaded_stop_words = joblib.load('../../09_Otros/stop_words_list.pkl')

# Aplicar CountVectorizer a nuevos datos
nuevas_caracteristicas = loaded_cv.transform(df.nombre_auto)
nuevas_columnas = pd.DataFrame.sparse.from_spmatrix(nuevas_caracteristicas, columns=loaded_cv.get_feature_names_out())
df = pd.concat([nuevas_columnas,df],axis=1)
df.drop(columns='nombre_auto',inplace= True)

#variables a imputar
var_imputar_mediana = ['caballos']

#función para imputar por la mediana
df['caballos'] = df.caballos.fillna(int(df.caballos.median()))

## ESTRUCTURA DEL DATASET

### VARIABLES FINALES

In [38]:
variables_finales = ['peso',
                    'caballos',
                    'aceleracion',
                    'origen',
                    'any_modelo',
                    'colt',
                    'rabbit',
                    'chevette',
                    'oldsmobile',
                    'grand',
                    'amc',
                    'malibu',
                    'sw',
                    'aspen',
                    'mazda',
                    'classic',
                    'volkswagen',
                    'carlo',
                    'datsun',
                    'matador',
                    'subaru',
                    'gl',
                    'concord',
                    'landau',
                    'corona',
                    '504',
                    'volvo',
                    'impala']

### ELIMINAR DUPLICADOS

In [39]:
df.drop_duplicates(inplace=True)

### ELIMINAR NULOS

In [40]:
df.dropna(thresh=3, inplace= True)

### PARA X e Y

In [41]:
x = df[variables_finales].copy()
y = df.mpg.copy()

## CREAR EL PIPELINE

In [42]:
### ONE HOT ENCODER
var_ohe = ['any_modelo','origen']
ohe = OneHotEncoder(sparse_output= False, handle_unknown= 'ignore')

### CON YEO-JOHNSON
var_yeo = ['caballos','peso','aceleracion']
yeo = PowerTransformer(method='yeo-johnson')

var_mms = ['peso','caballos','aceleracion']
mms = MinMaxScaler()
df_mms = mms.fit_transform(x[var_mms])

### CREAR EL PIPE DEL PREPROCESAMIENTO

#### Crear el column transformer

In [43]:
ct = make_column_transformer(
    (ohe, var_ohe),
    (yeo, var_yeo),
    (mms, var_mms) ,
    remainder= 'drop'
)

## INSTANCIAR EL MODELO

In [48]:
modelo = pd.read_pickle('../../04_Modelos/V0_pipe_ejecucion.pickle')

#### Crear el pipe final de entrenamiento

In [45]:
pipe_entrenamiento = make_pipeline(ct, modelo)

#### Guardar el pipe final de entrenamiento

In [46]:
# Guardar pipe de entrenamiento
nombre_pipe_entrenamiento = 'pipe_entrenamiento.pickle'
ruta_pipe_entrenamiento = '../../04_Modelos/' + nombre_pipe_entrenamiento

with open (ruta_pipe_entrenamiento, mode= 'wb') as file:
    cloudpickle.dump(pipe_entrenamiento, file)

### Entrenar el pipe final de ejecución

In [52]:
pipe_ejecucion = pipe_entrenamiento.fit(x,y)

### Guardar el pipe final de ejecución

In [53]:
nombre_pipe_ejecucion = 'pipe_ejecucion.pickle'
ruta_pipe_ejecucion = '../../04_Modelos/' + nombre_pipe_ejecucion

with open (ruta_pipe_ejecucion, mode= 'wb') as file:
    cloudpickle.dump(pipe_ejecucion, file)